In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Basic NLP EDA (на Pandas + Matplotlib)**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Загрузка данных
df = pd.read_csv('your_file.csv')  # Убедись, что в файле есть колонки: text, label

# 📌 Общая информация
print("📊 Размер:", df.shape)
print("\n🔍 Пропуски:\n", df.isnull().sum())
print("\n📄 Примеры:\n", df.sample(3))

# 🎯 Распределение классов
plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df)
plt.title('Распределение классов')
plt.show()

# 📏 Длина текстов
df['text_len'] = df['text'].apply(lambda x: len(str(x).split()))
plt.figure(figsize=(6,4))
sns.histplot(df['text_len'], bins=30)
plt.title('Распределение длины текстов (в словах)')
plt.xlabel('Количество слов')
plt.show()

# ☁️ Wordcloud по каждому классу
for label in df['label'].unique():
    text = ' '.join(df[df['label']==label]['text'].astype(str))
    tokens = [word for word in text.lower().split() if word not in stop_words and word not in string.punctuation]
    cloud = WordCloud(width=600, height=400, background_color='white').generate(' '.join(tokens))
    plt.figure(figsize=(7, 5))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Частые слова для label = {label}')
    plt.show()


**📈 4. Time Series (временные ряды)**

In [ ]:
df = pd.read_csv("time_data.csv", parse_dates=["date"])
df = df.sort_values("date")

# ⏳ Временной тренд
plt.plot(df['date'], df['value'])
plt.title("Тренд со временем")
plt.xlabel("Дата")
plt.ylabel("Значение")
plt.xticks(rotation=45)
plt.grid()
plt.show()

# 🔁 Сезонность
df['month'] = df['date'].dt.month
sns.boxplot(x='month', y='value', data=df)
plt.title("Сезонность по месяцам")
plt.show()


**Time series с категориями**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка
df = pd.read_csv("data.csv", parse_dates=['date'])

# ⏳ Общий тренд по всем категориям
plt.figure(figsize=(10, 5))
df_grouped = df.groupby('date')['value'].sum()
plt.plot(df_grouped.index, df_grouped.values)
plt.title('Общий тренд по всем категориям')
plt.grid()
plt.show()

# 🧩 Индивидуальные линии по категориям
plt.figure(figsize=(12, 6))
for cat in df['category'].unique():
    data = df[df['category'] == cat]
    grouped = data.groupby('date')['value'].sum()
    plt.plot(grouped.index, grouped.values, label=cat)

plt.legend()
plt.title('Тренды по категориям')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.grid()
plt.show()

# 📊 Boxplot: сезонность внутри года
df['month'] = df['date'].dt.month
plt.figure(figsize=(10, 5))
sns.boxplot(x='month', y='value', hue='category', data=df)
plt.title('Сезонность по месяцам и категориям')
plt.show()


**🧪 Базовый EDA для CV (на ImageFolder)**

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import random
from collections import Counter

data_dir = "train"  # структура: train/class_name/image.jpg

# 📊 Распределение по классам
classes = os.listdir(data_dir)
class_counts = {cls: len(os.listdir(os.path.join(data_dir, cls))) for cls in classes}

plt.bar(class_counts.keys(), class_counts.values())
plt.xticks(rotation=45)
plt.title("Количество изображений по классам")
plt.show()

# 🖼️ Случайные изображения по классам
for cls in random.sample(classes, min(5, len(classes))):
    img_path = os.path.join(data_dir, cls, random.choice(os.listdir(os.path.join(data_dir, cls))))
    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Class: {cls}")
    plt.show()

# 📐 Размеры изображений
shapes = []
for cls in classes:
    sample_imgs = os.listdir(os.path.join(data_dir, cls))[:10]
    for img_name in sample_imgs:
        img_path = os.path.join(data_dir, cls, img_name)
        try:
            img = Image.open(img_path)
            shapes.append(img.size)
        except:
            continue

sizes_df = pd.DataFrame(shapes, columns=["width", "height"])
sns.histplot(sizes_df["width"], bins=20, color="skyblue", label="Width")
sns.histplot(sizes_df["height"], bins=20, color="orange", label="Height")
plt.legend()
plt.title("Распределение размеров изображений")
plt.show()


**🧪 EDA-шаблон для CV из CSV**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import random

# Загрузка данных
df = pd.read_csv("data.csv")

# 📊 Баланс классов
plt.figure(figsize=(8,4))
sns.countplot(y="label", data=df, order=df['label'].value_counts().index)
plt.title("Распределение классов")
plt.show()

# 🖼️ Визуализация случайных изображений по меткам
def show_images_by_class(df, label, n=3):
    samples = df[df['label'] == label].sample(n)
    plt.figure(figsize=(n*3, 3))
    for i, path in enumerate(samples['image_path']):
        img = Image.open(path)
        plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(label)
    plt.show()

for label in df['label'].value_counts().index[:5]:  # Покажем до 5 классов
    show_images_by_class(df, label)

# 📐 Распределение размеров изображений
widths, heights = [], []
for path in df['image_path'].sample(min(200, len(df))):  # 200 случайных изображений
    try:
        img = Image.open(path)
        widths.append(img.width)
        heights.append(img.height)
    except:
        continue

plt.figure(figsize=(10, 4))
sns.histplot(widths, bins=30, color="skyblue", label="Width")
sns.histplot(heights, bins=30, color="orange", label="Height")
plt.title("Распределение размеров изображений")
plt.legend()
plt.show()


**📦 Табличный EDA — универсальный шаблон**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("data.csv")  # Предположим, что target — целевая переменная

# 🧱 Общая структура
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())

# 🧪 Распределение целевой переменной
plt.figure(figsize=(6, 4))
if df['target'].nunique() <= 10:
    sns.countplot(x='target', data=df)
    plt.title("Классы (classification)")
else:
    sns.histplot(df['target'], bins=30)
    plt.title("Целевая переменная (regression)")
plt.show()

# 📊 Распределение числовых фич
num_cols = df.select_dtypes(include='number').drop('target', axis=1).columns
df[num_cols].hist(figsize=(12, 8), bins=30)
plt.suptitle("Гистограммы числовых признаков")
plt.show()

# 🔗 Корреляция
plt.figure(figsize=(10, 8))
corr = df[num_cols.tolist() + ['target']].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Корреляционная матрица")
plt.show()

# 🧩 Категориальные фичи
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    plt.figure(figsize=(6, 3))
    sns.countplot(y=col, data=df, order=df[col].value_counts().index)
    plt.title(f"Категориальный признак: {col}")
    plt.show()


📦 Почему это полезно:
Когда использовать	Зачем
🔍 EDA	Чтобы визуализировать в 2D
🧹 Много признаков	Убрать шум и мультиколлинеарность
🚀 Ускорить обучение моделей	Особенно в линейных или KNN/Clustering
🎯 Обработка коррелированных фичей	PCA превращает их в независимые оси
⚠️ Что не делает PCA

    Не сохраняет смысл исходных признаков

    Не использует целевую переменную (target) → не supervised

    Не подходит для категориальных признаков без кодирования

📐 Как работает PCA (в 3 шага)

    Центрируем данные (отнимаем среднее)

    Находим ковариационную матрицу

    Вычисляем собственные векторы и значения
    → они дают направления (компоненты) и вес (дисперсию)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. PCA до 2 компонент
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 3. Визуализация
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.title('PCA визуализация')
plt.show()

# 4. Сколько объясняет каждая компонента?
print(pca.explained_variance_ratio_)  # например: [0.6, 0.3]
